In [ ]:
import glob
import os
import pandas as pd 
import re
from pandas import Series, DataFrame
from textblob import TextBlob, Word
import nltk
from nltk.util import ngrams
import re
nltk.download('stopwords') 
from nltk.corpus import stopwords
from nltk.util import ngrams
import string
from nltk.probability import FreqDist
import gensim
from gensim import models # For TF-IDF, LDA
import swifter # Makes applying to datframe as fast as vectorizing
import numpy as np
import collections
import csv

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline

# LDA Visualization
import pyLDAvis
import pyLDAvis.gensim 


What is happening here:
- Per time window
    1. Save all vocab + frequencies
    2. Save all blacklisted/retained vocab + frequencies ---> not done yet
- Within each window
    1. Pre-process:
        - Clean (remove special characters, ect.)
        - Remove stopwords
        - Lemmatize based on POS
        - Add n-grams
        - Save tokens to new columns
    2. Get vocab frequency for each article:
        - Save token frequencies to new column
    3. Merge all vocab tokens to get blacklist:
        - Save all blacklisted and retained tokens in new text/csv files respectively
        - Save all blacklisted and retained tokens in new columns respectively
    4. Save new article CSV files with added columns

In [ ]:
# ------------------- FUNCTIONS ---------------------------------------------------------

# Pre-procesing function

stop_words = stopwords.words('english')

stop_words.extend(['chars', 'char']) # Add from blacklist

stop_words.extend(['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday'])



# stop_words.extend(['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december'])
# ---> Not removing month names, some of them have different meanings 

stop_words.extend(['get', 'say', 'gmt', 'dont', 'make', 'want', 'also', 
                   'take', 'since', 'tell', 'like', 'could', 'would', 'etc',
                   'should', 'jsfjsdgetelementsbytagnames0p', 'functiondsidvar']) # Adding from LDA topics 

def preprocess_text(doc, n_grams = True):
    
    """
    Pre-processing using TextBlob: 
    tokenizing, converting to lower-case, and lemmatization based on POS tagging, 
    removing stop-words, and retaining tokens greater than length 2
    
    We can also choose to include n_grams (n = 1,2,3,4) in the final output
    
    Argument(s): 'doc' - a string of words or sentences.
                 'n_grams' - boolean: if set to 'True', n_grams are included
    
    Output: 'reuslt_singles' - a list of pre-processed tokens (individual words) of each sentence in 'doc'
            'result_ngrams' - a list of pre-processed tokens (including n-grams) of each sentence in 'doc'
    
    """
    
    doc = re.sub(r'([^\w\s-]|_)+', ' ',  doc) # ---> special characters
    doc = re.sub(r'http\S+', ' ', doc) # ---> hyperlinks
    doc = re.sub(r'Let friends in your social network know what you are reading about .*? Please read the rules before joining the discussion.', ' ', doc) # ---> 
    doc = re.sub(r'___ For more .*? This material may not be published, broadcast, rewritten or redistributed.', ' ', doc) # --->
    doc = re.sub(r'___ For more .*? by Automated Insights,  using data from STATS LLC, ', ' ', doc) # --->
    doc = re.sub(r'For more AP.*? by Automated Insights,  using data from STATS LLC, ', ' ', doc) # --->

    blob = TextBlob(doc).lower() 
    
    result_singles = []
    tag_dict = {"J": 'a', # Adjective
                "N": 'n', # Noun
                "V": 'v', # Verb
                "R": 'r'} #  Adverb
    
    # For all other types of parts of speech (including those not classified at all) 
    # the tag_dict object maps to 'None'
    # the method w.lemmatize() defaults to 'Noun' as POS for those classified as 'None'
    
    
#     bigrams = blob.ngrams(n = 2)    # ---> not so useful, includes stopwords
#     trigrams = blob.ngrams(n = 3)
#     fourgrams = blob.ngrams(n = 4)

    for sent in blob.sentences:

        words_and_tags = [(w, tag_dict.get(pos[0])) for w, pos in sent.tags]
        lemmatized_list = [w.lemmatize(tag) for w, tag in words_and_tags]

        for i in range(len(lemmatized_list)):
            if lemmatized_list[i] not in stop_words and len(lemmatized_list[i].lower()) > 2:
                result_singles.append(lemmatized_list[i].lower())
                
    result_ngrams = result_singles + [' '.join(x) for x in ngrams(result_singles, 2)] + [' '.join(x) for x in ngrams(result_singles, 3)] + [' '.join(x) for x in ngrams(result_singles, 4)]

#     + [' '.join(i) for i in bigrams] + [' '.join(i) for i in trigrams] + [''.join(i) for i in fourgrams]  # if we want to include stopwords          
    
    if n_grams == True:
        result = result_ngrams
    else:
        result = result_singles
    
    return result


In [ ]:
# doc1 = "Natural Language Processing (NLP) is an area of growing attention due to increasing number of applications like chatbots, machine translation etc. In some ways, the entire revolution of intelligent machines in based on the ability to understand and interact with humans. I have been exploring NLP for some time now.  My journey started with NLTK library in Python, which was the recommended library to get started at that time. NLTK is a perfect library for education and research, it becomes very heavy and tedious for completing even the simple tasks."
# doc2 = "This one contains multiple symbols.... like this: +327 -12, 63*... and this too %43 (jkfdjfdf). It just goes on to have more stuff $$ #"
# doc3 = "html,https://static01.nyt.com/images/2019/03/28/realestate/02fix1/oakImage-1553807804790-facebookJumbo.jpg,2019-04-02T00:00:00Z,Alexa, Siri and Google Assistant can all make controlling a smart home with voice commands easy, but the idea of adding smart speakers with microphones that are always listening for the next command makes some people uneasy. Indeed, mistakes have been made. … [+1224 chars], Advertisement Supported by THE FIX You no longer need to renovate — or spend a lot of money — to incorporate smart technology into your home. Here’s how to do it in a few quick steps. By Tim McKeough Just a few years ago, creating a smart home with coordinated controls for lighting, motorized shades, multiroom audio, and heating and cooling required an expensive, comprehensive home-automation system and a renovation.  When you used to think about these systems, it was very much high-end,” said Mark Spates, a senior product manager for Nest and Google Home in Mountain View, Calif. “It was a luxury. Today, many of those functions are accessible through affordable, consumer-friendly products. You can pick and choose the functions you want and install many of the products yourself"
# a = preprocess_text(doc1)
# b = preprocess_text(doc2)
# c = preprocess_text(doc3)


In [ ]:
# Function for word frequencies

def get_frequency(processed_text_list): 

    """
    Using a built-in NLTK function that generates tuples
    We get the frequency distribution of all words/n-grams in a tokenized list
    We get the proportion of the the token as a fraction of the total corpus size  ----> N/A
    We also sort these frequencies and proportions in descending order in a dictionary object ----> N/A
    
    Argument(s): 'processed_text_list' - A list of pre-processed tokens
    
    Output(s): freq_dict - A dictionary of tokens and their respective frequencies in descending order
    """
               #prop_dict - A dictionary of tokens and their respective proportions as a fraction of the total corpus
               #combined_dict - A dictionary whose values are both frequencies and proportions combined within a list
    #"""

    word_frequency = FreqDist(word for word in processed_text_list)

#     sorted_counts = sorted(word_frequency.items(), key = lambda x: x[1], reverse = True)
#     freq_dict = dict(sorted_counts)
    freq_dict = dict(word_frequency)
#     prop_dict = {key : freq_dict[key] * 1.0 / sum(freq_dict.values()) for key, value in freq_dict.items()}
#     combined_dict = {key : [freq_dict[key], freq_dict[key] * 1.0 / sum(freq_dict.values())] for key, value in freq_dict.items()}

    return freq_dict #, prop_dict, combined_dict


In [ ]:
def merge_vocab_dictionary(vocab_column):
    
    """
    Takes any number of token frequency dictionaries and merges them while summing 
    the respective frequencies and then calculates the proportion of the the tokens 
    as a fraction of the total corpus size and saves to text and CSV files
    
    
    Argument(s): vocab_column - A column of dictionary objects
                 
    Output(s): merged_combined_dict - A list object containing the frequencies of all
               merged dictionary tokens along with their respective proportions
    """
    
    merged_freq_dict = {}
    for dictionary in vocab_column:
        for key, value in dictionary.items():  # d.items() in Python 3+
            merged_freq_dict.setdefault(key, []).append(value)


    for key, value in merged_freq_dict.items():
        merged_freq_dict[key] = sum(value)
    
    total_sum = sum(merged_freq_dict.values())
#     merged_prop_dict = {key : merged_freq_dict[key] * 1.0 / total_sum for key, value in merged_freq_dict.items()}
    merged_combined_dict = {key : [merged_freq_dict[key], (merged_freq_dict[key] * 1.0 / total_sum)] for key, value in merged_freq_dict.items()}
     
    return merged_combined_dict
    

In [ ]:
def blacklist(freq_prop_dict, name = 'test_file', less_than = 5, greater_than_prop = 0.4):
    
    """
    We use the 
    
    Argument(s): 'freq_prop_dict' - a dictionary 
    
    Output: 'blacklist' - a dictionary of blacklisted tokens from a given dictionary
            'retained' - a dictionary of
    """

    blacklist = {key : value for key, value in freq_prop_dict.items() if (value[0] < less_than or value[1] > greater_than_prop)}
    retained = {key : value for key, value in freq_prop_dict.items() if (value[0] >= less_than and value[1] <= greater_than_prop)}
        
    return blacklist, retained

# ---------- Other Approaches -----------

#     blacklist = []
#     retained = []
#     for key, value in merged_dict.items():
#     if value[0] < 5 and value[1] > 0.4:
#         blacklist.append(key)
#     else:
#         retained.append(key)
    
    
    
#    # The gensim approach
#    # Outputs the tokens that have been filtered out using the gensim dictionary
#     for token in processed_tokens_list:
#         if token not in short_text_dictionary.token2id.keys():
#             blacklist.append(token)
            

In [ ]:
def dictionary_saver(save_dictionary, save_directory, save_name  = 'test_dictionary', text_file = True, csv_file = True):
    
    """
    We use the 
    
    Argument(s): 'freq_prop_dict' - a dictionary 
    
    Output: a saved text file containing all the dictionary elements
            a saved CSV file containing all the dictionary elements
    """
    
    if text_file == True:
        
        txt_save_name = save_directory + save_name +'.txt'
    
        with open(txt_save_name, 'w+', encoding = "utf-8") as txt_file:
            for key, value in save_dictionary.items():
                txt_file.write("%s\n" % f"{key}: {value}")
                
    if csv_file == True:
        
        csv_save_name = save_directory + save_name + '.csv'
        
        with open(csv_save_name, 'w+') as csv_file:
            csv_file.write("%s,%s,%s\n"%('Token', 'Frequency', 'Proportion'))
            for key, value in save_dictionary.items():
                csv_file.write("%s,%s,%s\n"%(key, value[0], value[1]))
                
    return 'Files successfully saved'
    

In [ ]:

def filter_after_preprocess(processed_tokens, vocabulary_dict):
    
    """
    We use the 
    
    Argument(s): processed_tokens  -
                 vocabulary_dict -
    
    Output: filtered 
    """
    filtered = []

    for token in processed_tokens:
        if token in vocabulary_dict.keys():
            filtered.append(token)

    return filtered



In [ ]:
# We shall first import all the CSV files that we have on file
# We store all these files in a list


# directory = "/home/sdbookhu/FullNews/"

rootdir_glob = '/home/sdbookhu/FullNews/**/*'

all_articles = [f for f in glob.iglob('FullNews/**/*.csv', recursive=True) if os.path.isfile(f)]
all_articles = sorted(all_articles, key = lambda file: file[-14:-4])

print("The total number of CSV files is: " + str(len(all_articles)))  

# Now, we will read in files from the list based on our sliding window

In [ ]:
window = 2 # This is for a sliding window of size 1

full_vocab_window_1 = {}
blacklist_window_1 = {}
retained_window_1 = {}

main_vocab_list = [full_vocab_window_1, blacklist_window_1, retained_window_1]

In [ ]:
# for i in range(0, (len(all_articles) - window - 1)):

for i in range(11, 20):
    
    df = pd.concat(map(pd.read_csv, all_articles[i:i+ window]), axis=0, ignore_index=True)
    name = 'window_' + str(window) + all_articles[i][-14:-4]
    print(name)
    
    path = '/home/sdbookhu/' + name + '/'
    os.makedirs(path, exist_ok = True)
    save_path = path
    print(save_path)
    # # ------------------- PRE-PROCESS ARTICLES ---------------------------------------------------------

    print("STEP 1")

    data_text = df.copy() # Not altering the original

    data_text['old_index'] = data_text.index

    """
    We have this step so that we can locate which specific articles we have retained/deleted
    The index gets altered when we remove duplicates and missing articles
    """

    print("Total number of short and long articles is: ", len(data_text))

    data_text = data_text.dropna(subset = ["content", "full-content"]) # Not all articles have any content available
    print("Total number of short and long articles after dropping blank ones: ", len(data_text))

    data_text = data_text.drop_duplicates(subset=["title"], keep = 'last') # ---> Read below

    """
    NOTE: The argument 'keep' changes the type of articles retained

    keep = 'last' ---> Ensures that the latest article is retained 
           (there might be changes in the content over time - as indicated by higher 
           retention of aritcles when 'url' is included in the subset.)
           However, this leads to more AP sources

    keep = 'first' ---> Retains original source more often.

    # We have many repeating articles
    # Dropping duplicates based on article title and description
    # Including "source_id" as a duplicate subset leads to fewer drops --> same article, different sources
    # Keeping least recent article (based on 'publishedAt')

    """

    print("Total number of unique short and long articles is: ", len(data_text))

    print("STEP 2")
    data_text['short_processed_text'] = data_text['content'].swifter.apply(preprocess_text)
    # Then Long
    data_text['long_processed_text'] = data_text['full-content'].swifter.apply(preprocess_text)

    print("STEP 3")

    data_text['short_all_frequencies'] = data_text['short_processed_text'].swifter.apply(get_frequency)
    data_text['long_all_frequencies'] = data_text['long_processed_text'].swifter.apply(get_frequency)

    print("STEP 3 COMPLETE")

    print("STEP 4")

    short_full_vocab = merge_vocab_dictionary(data_text['short_all_frequencies'])
    print("STEP 4 COMPLETE")
    print("STEP 5")
    long_full_vocab = merge_vocab_dictionary(data_text['long_all_frequencies'])

    print("STEP 5 COMPLETE")
    print("STEP 6")
    short_blacklist_dict, short_retained_dict = blacklist(short_full_vocab)
    print("STEP 7")
    long_blacklist_dict, long_retained_dict = blacklist(long_full_vocab)
    print("STEP 7 COMPLETE")
    
#     --------------------------------------------------------------

    stop_words.extend(list(long_blacklist_dict.keys())) # ----> adding newly blacklisted words to stopwords
    
#     --------------------------------------------------------------

    print("STEP 8")
    
    save_directory = save_path
    save_name = name + "_short_full_vocab"   
    dictionary_saver(short_full_vocab, save_directory, save_name  = save_name, text_file = True, csv_file = True)
    
    save_name = name + "_long_full_vocab"   
    dictionary_saver(long_full_vocab, save_directory, save_name  = save_name, text_file = True, csv_file = True)
    
    save_name = name + "_short_blacklist"   
    dictionary_saver(short_blacklist_dict, save_directory, save_name  = save_name, text_file = True, csv_file = True)
    
    save_name = name + "_long_blacklist"   
    dictionary_saver(long_blacklist_dict, save_directory, save_name  = save_name, text_file = True, csv_file = True)
    
    save_name = name + "_short_retained"   
    dictionary_saver(short_retained_dict, save_directory, save_name  = save_name, text_file = True, csv_file = True)
    
    save_name = name + "_long_retained"   
    dictionary_saver(long_retained_dict, save_directory,  save_name  = save_name, text_file = True, csv_file = True)
    
    print("STEP 8 COMPLETE")
    
    print("STEP 9")
    
    vocabulary_dict = short_retained_dict
    data_text['short_retained']  = data_text['short_processed_text'].swifter.apply(filter_after_preprocess, args=(vocabulary_dict,))
    
    vocabulary_dict = long_retained_dict
    data_text['long_retained']  = data_text['long_processed_text'].swifter.apply(filter_after_preprocess, args=(vocabulary_dict,))
    
    vocabulary_dict =  short_blacklist_dict
    data_text['short_blacklist'] = data_text['long_processed_text'].swifter.apply(filter_after_preprocess, args=(vocabulary_dict,))
    
    vocabulary_dict = long_blacklist_dict
    data_text['long_blacklist'] = data_text['long_processed_text'].swifter.apply(filter_after_preprocess, args=(vocabulary_dict,))
    
    print("STEP 9 COMPLETE")
    
    print("STEP 10")

    data_old_index = data_text['old_index']
    data_url = data_text['url']
    data_source = data_text['source_name']
    data_publish_time = data_text['publishedAt']
    data_title = data_text['title']
    data_description = data_text['description']
    data_short_content = data_text['content']
    data_full_content = data_text['full-content']
    data_short_all_tokens = data_text['short_processed_text']
    data_long_all_tokens = data_text['long_processed_text']
    data_short_all_frequencies = data_text['short_all_frequencies']
    data_long_all_frequencies = data_text['short_all_frequencies']
    data_short_blacklist = data_text['short_blacklist']
    data_long_blacklist = data_text['long_blacklist']
    data_short_retained = data_text['short_retained']
    data_long_retained = data_text['long_retained']
    
    
    """
    Use the script below to make changes to the CSV file and save as a different CSV file 
    """
    
    news_old_index = []
    news_url = []
    news_source = []
    news_publish_time = []
    news_title = []
    news_description = []
    news_short_content = []
    news_full_content = []
    short_all_processed_tokens = []
    long_all_processed_tokens = []
    short_all_frequencies = []
    long_all_frequencies = []
    short_blacklist = []
    long_blacklist = []
    short_retained = []
    long_retained = []
    

    for i in range(0, len(data_text)):

        news_old_index.append(data_old_index.iloc[i])
        news_url.append(data_url.iloc[i])
        news_source.append(data_source.iloc[i])
        news_publish_time.append(data_publish_time.iloc[i])
        news_title.append(data_title.iloc[i])
        news_description.append(data_description.iloc[i])
        news_short_content.append(data_short_content.iloc[i])
        news_full_content.append(data_full_content.iloc[i])
        short_all_processed_tokens.append(data_short_all_tokens.iloc[i])
        long_all_processed_tokens.append(data_long_all_tokens.iloc[i])
        short_all_frequencies.append(data_short_all_frequencies.iloc[i]) 
        long_all_frequencies.append(data_long_all_frequencies.iloc[i]) 
        short_retained.append(data_short_retained.iloc[i])
        long_retained.append(data_long_retained.iloc[i])
        short_blacklist.append(data_short_blacklist.iloc[i])
        long_blacklist.append(data_long_blacklist.iloc[i])


    news_file_df = DataFrame({
                    'old_index': news_old_index,
                    'url': news_url,
                    'source': news_source,
                    'publishedAt': news_publish_time,
                    'title': news_title,
                    'description': news_description,
                    'content': news_short_content,
                    'full-content': news_full_content,
                    'short_all_tokens': short_all_processed_tokens,
                    'long_all_tokens': long_all_processed_tokens,
                    'short_all_frequencies':  short_all_frequencies,
                    'long_all_frequencies':  long_all_frequencies,
                    'short_retained_tokens': short_retained,
                    'long_retained_tokens': long_retained,
                    'short_blacklist_tokens': short_blacklist,
                    'long_retained_blacklist': long_blacklist
                    })

    news_file_df = news_file_df[['old_index','url', 'source', 'publishedAt', 'title',
                                 'description', 'content', 'full-content',
                                 'short_all_tokens', 'long_all_tokens', 'short_all_frequencies',
                                 'long_all_frequencies','short_retained_tokens', 'long_retained_tokens',
                                 'short_blacklist_tokens', 'long_blacklist_tokens']]

    save_directory = save_path + 'AllContent_' + name + '.csv'
    
    news_file_df.to_csv(save_directory, index = None, header=True, encoding='utf-8')
    

    print("STEP 10 COMPLETE")
    print(" ")
    print("Loop complete")
    print("Next")
    print(" ")
    print(" ")



In [ ]:
# # Saving all vocabulary by window to main vocabulary file
# # Can be done separately,  saves time.

# full_vocab_window_1 = {}
# blacklist_window_1 = {}
# retained_window_1 = {}

# main_vocab_list = [full_vocab_window_1, blacklist_window_1, retained_window_1] # --->  call from within files.

# vocab_dicts = [short_full_vocab, long_full_vocab, short_blacklist, short_retained, long_blacklist, long_retained] # --->  call from within files.

# for main_dict in main_vocab_list:
#     for dictionary in vocab_dicts:
#         for key, value in dictionary.items():  
#             main_dict.setdefault(key, []).append(value)

#     for key, value in main_dict.items():
#         main_dict[key] = sum(value)